In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import os


In [2]:
data_dir = '/Users/subhadyutirath/Documents/FitSense/imgs'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

batch_size = 32


In [ ]:
dataset = ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 3)  # 3 classes
model = model.to(device)


/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/subhadyutirath/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:08<00:00, 5.23MB/s]


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [16]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1, Loss: 0.0014
Epoch 2, Loss: 0.0020
Epoch 3, Loss: 0.0012
Epoch 4, Loss: 0.0019
Epoch 5, Loss: 0.0015
Epoch 6, Loss: 0.0009
Epoch 7, Loss: 0.0011
Epoch 8, Loss: 0.0022
Epoch 9, Loss: 0.0011
Epoch 10, Loss: 0.0008
Epoch 11, Loss: 0.0013
Epoch 12, Loss: 0.0007
Epoch 13, Loss: 0.0033
Epoch 14, Loss: 0.0010
Epoch 15, Loss: 0.0009
Epoch 16, Loss: 0.0021
Epoch 17, Loss: 0.0011
Epoch 18, Loss: 0.0007
Epoch 19, Loss: 0.0007
Epoch 20, Loss: 0.0007
Epoch 21, Loss: 0.0008
Epoch 22, Loss: 0.0010
Epoch 23, Loss: 0.0006
Epoch 24, Loss: 0.0006
Epoch 25, Loss: 0.0006
Epoch 26, Loss: 0.0006
Epoch 27, Loss: 0.0008
Epoch 28, Loss: 0.0008
Epoch 29, Loss: 0.0018
Epoch 30, Loss: 0.0006
Epoch 31, Loss: 0.0010
Epoch 32, Loss: 0.0015
Epoch 33, Loss: 0.0007
Epoch 34, Loss: 0.0006
Epoch 35, Loss: 0.0006
Epoch 36, Loss: 0.0012
Epoch 37, Loss: 0.0006
Epoch 38, Loss: 0.0006
Epoch 39, Loss: 0.0006
Epoch 40, Loss: 0.0006
Epoch 41, Loss: 0.0005
Epoch 42, Loss: 0.0011
Epoch 43, Loss: 0.0008
Epoch 44, Loss: 0.00

In [17]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Validation Accuracy: 80.00%


In [ ]:
class_names = ['casuals', 'formal', 'sports'] 

In [18]:
from PIL import Image

def predict(image_path):
    model.eval()
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

# Example usage
print(predict("/Users/subhadyutirath/Documents/FitSense/51EfYoRALoL.jpg"))


sports


In [23]:
torch.save(model.state_dict(), 'fitsense.pth')